In [2]:
#!/usr/bin/env python3
import os
import math
import requests
import pandas as pd
import numpy as np
import datetime
import io
import matplotlib.pyplot as plt

# ---------------------------------------------
# Configuration: меняем только список активов
# ---------------------------------------------
ASSETS = ["omg","doge", "neo"]  # используемые тикеры CoinMetrics
HISTORY_DAYS = 90               # окно в днях

# Метрики, которые будем запрашивать
METRICS = [
    "AdrActCnt", "TxTfrValAdjNtv", "CapMrktCurUSD", "CapRealUSD",
    "FlowInExNtv", "FlowOutExNtv", "BlkSizeMeanByte"
]

def fetch_coinmetrics_data(asset: str, days: int) -> pd.DataFrame:
    end_date   = datetime.datetime.utcnow().strftime("%Y-%m-%d")
    start_date = (datetime.datetime.utcnow() - datetime.timedelta(days=days)).strftime("%Y-%m-%d")
    url = (
        "https://community-api.coinmetrics.io/v4/timeseries/asset-metrics"
        f"?assets={asset}&metrics={','.join(METRICS)}"
        f"&start_time={start_date}&end_time={end_date}"
        "&frequency=1d&format=csv"
    )
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    df = pd.read_csv(io.StringIO(resp.text))
    df['time'] = pd.to_datetime(df['time'])
    # дополнительные колонки
    df['NVT'] = np.where(df['TxTfrValAdjNtv'] > 0,
                         df['CapMrktCurUSD'] / df['TxTfrValAdjNtv'], np.nan)
    df['MVRV'] = np.where(df['CapRealUSD'] > 0,
                          df['CapMrktCurUSD'] / df['CapRealUSD'], np.nan)
    df['ExchNetFlow'] = df['FlowInExNtv'] - df['FlowOutExNtv']
    return df

def fetch_price_data(asset: str, days: int) -> pd.DataFrame:
    end_date   = datetime.datetime.utcnow().strftime("%Y-%m-%d")
    start_date = (datetime.datetime.utcnow() - datetime.timedelta(days=days)).strftime("%Y-%m-%d")
    url = (
        "https://community-api.coinmetrics.io/v4/timeseries/asset-metrics"
        f"?assets={asset}&metrics=PriceUSD"
        f"&start_time={start_date}&end_time={end_date}"
        "&frequency=1d&format=csv"
    )
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    df = pd.read_csv(io.StringIO(resp.text))
    df['time'] = pd.to_datetime(df['time'])
    return df

def store_data(df: pd.DataFrame, filename: str):
    if os.path.exists(filename):
        old = pd.read_csv(filename)
        old['time'] = pd.to_datetime(old['time'])
        combined = pd.concat([old, df]).drop_duplicates(subset='time', keep='last')
        combined.sort_values('time', inplace=True)
        combined.to_csv(filename, index=False)
    else:
        df.to_csv(filename, index=False)
    print(f"[{filename}] saved, rows={len(df)}")

def plot_metrics(asset: str, cm_df: pd.DataFrame, price_df: pd.DataFrame):
    cm_df    = cm_df.sort_values('time')
    price_df = price_df.sort_values('time')

    # Список потенциальных панелей (метка, заголовок, флаг is_flow)
    all_panels = [
        ("ExchNetFlow",    "Exchange Net Flow",    True),
        ("NVT",            "NVT Ratio",            False),
        ("MVRV",           "MVRV Ratio",           False),
        ("AdrActCnt",      "Active Addresses",     False),
        ("BlkSizeMeanByte","Avg Block Size",       False),
    ]

    # Оставляем только те панели, для которых есть данные
    panels = []
    for col, title, is_flow in all_panels:
        if is_flow:
            if {"FlowInExNtv", "FlowOutExNtv", "ExchNetFlow"}.issubset(cm_df.columns):
                panels.append((col, title, is_flow))
        else:
            if col in cm_df.columns:
                panels.append((col, title, is_flow))

    if not panels:
        print(f"No available on-chain metrics to plot for {asset.upper()}.")
        return

    # Параметры сетки
    n = len(panels)
    ncols = min(3, n)
    nrows = math.ceil(n / ncols)

    fig, axes = plt.subplots(nrows, ncols, figsize=(6*ncols, 4*nrows), squeeze=False)
    fig.suptitle(f"{asset.upper()} On-Chain Metrics", fontsize=18)

    def overlay_price(ax):
        y0, y1 = ax.get_ylim()
        p0, p1 = price_df['PriceUSD'].min(), price_df['PriceUSD'].max()
        scaled = y0 + (price_df['PriceUSD'] - p0) * (y1 - y0) / (p1 - p0)
        ax.plot(price_df['time'], scaled, '--', alpha=0.6, label='PriceUSD')

    # Рисуем доступные панели
    for idx, (col, title, is_flow) in enumerate(panels):
        row, col_idx = divmod(idx, ncols)
        ax = axes[row][col_idx]

        if is_flow:
            t = cm_df['time']
            ax.plot(t, cm_df['FlowInExNtv'],  ':', alpha=0.4, label='Inflow')
            ax.plot(t, cm_df['FlowOutExNtv'], ':', alpha=0.4, label='Outflow')
            ax.plot(t, cm_df['ExchNetFlow'],  '-', alpha=0.5, label='Net Continuum')
            ax.plot(t, cm_df['ExchNetFlow'].where(cm_df['ExchNetFlow']>=0),
                    linewidth=2, label='+')
            ax.plot(t, cm_df['ExchNetFlow'].where(cm_df['ExchNetFlow']< 0),
                    linewidth=2, label='–')
        else:
            ax.plot(cm_df['time'], cm_df[col], linewidth=2, label=title)

        ax.set_title(title)
        overlay_price(ax)
        ax.legend(loc='best')
        ax.grid(True)
        ax.set_xlabel('Date')

    # Скрываем лишние оси
    total_axes = nrows * ncols
    for idx in range(len(panels), total_axes):
        row, col_idx = divmod(idx, ncols)
        axes[row][col_idx].axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    out = f"onchain-{asset.upper()}.png"
    plt.savefig(out, dpi=300)
    plt.close(fig)
    print(f"[Plot saved] {out}")

def main():
    for asset in ASSETS:
        cm = fetch_coinmetrics_data(asset, HISTORY_DAYS)
        pr = fetch_price_data(asset, HISTORY_DAYS)
        store_data(cm, f"{asset}_onchain_daily.csv")
        store_data(pr, f"{asset}_price_daily.csv")
        if not cm.empty and not pr.empty:
            plot_metrics(asset, cm, pr)

if __name__ == "__main__":
    main()



[omg_onchain_daily.csv] saved, rows=90
[omg_price_daily.csv] saved, rows=90
[Plot saved] onchain-OMG.png
[doge_onchain_daily.csv] saved, rows=90
[doge_price_daily.csv] saved, rows=90
[Plot saved] onchain-DOGE.png
[neo_onchain_daily.csv] saved, rows=90
[neo_price_daily.csv] saved, rows=90
[Plot saved] onchain-NEO.png
